In [1]:
import boto3 
import json
import time

In [7]:
CLIENT = boto3.client("athena")

DATA_CATALOG = "AwsDataCatalog"
DATABASE_NAME = "abraseldatalake_v1"
RESULT_OUTPUT_LOCATION = "s3://abrasel-datalake/query-results/"
TABLE_NAME = "cnpj_ativos"

In [5]:
def has_query_succeeded(execution_id):
    state = "RUNNING"
    max_execution = 5

    while max_execution > 0 and state in ["RUNNING", "QUEUED"]:
        max_execution -= 1
        response = CLIENT.get_query_execution(QueryExecutionId=execution_id)
        if (
            "QueryExecution" in response
            and "Status" in response["QueryExecution"]
            and "State" in response["QueryExecution"]["Status"]
        ):
            state = response["QueryExecution"]["Status"]["State"]
            if state == "SUCCEEDED":
                return True

        time.sleep(30)

    return False

In [8]:
def get_num_rows():
    query = f"SELECT COUNT(*) from {DATA_CATALOG}.{DATABASE_NAME}.{TABLE_NAME}"
    response = CLIENT.start_query_execution(
        QueryString=query,
        ResultConfiguration={"OutputLocation": RESULT_OUTPUT_LOCATION}
    )

    return response["QueryExecutionId"]


In [9]:
def get_query_results(execution_id):
    response = CLIENT.get_query_results(
        QueryExecutionId=execution_id
    )

    results = response['ResultSet']['Rows']
    return results

In [12]:
id_consulta = get_num_rows()


In [14]:
print(get_query_results(id_consulta))

[{'Data': [{'VarCharValue': '_col0'}]}, {'Data': [{'VarCharValue': '67665840'}]}]


In [5]:
linhas = 0
with open(r"C:\Users\ABRASEL NACIONAL\Documents\CNPJ_PROGRAMATICA\ESTABELECIMENTOSCSV\K3241.K03200Y0.D30114.csv", mode='r', encoding='ISO-8859-1', errors='ignore') as arq:
    #arq.seek(0, 2)
    #print(arq.tell())
    #print(len(arq.readline()))
    #linhas = arq.tell() // len(arq.readline())
    for linha in arq:
        linhas += 1
linhas

12374347

In [4]:
import requests
import pandas as pd
import json

df = pd.read_csv(r"C:\Users\ABRASEL NACIONAL\Downloads\cnaes.csv")
df.rename(columns={'Id':'Codigo Cnae'}, inplace=True)
df

,Codigo Cnae,Descrição,Código Longo
0,111301,CULTIVO DE ARROZ,0111-3/01
1,111302,CULTIVO DE MILHO,0111-3/02
2,111303,CULTIVO DE TRIGO,0111-3/03
3,111399,CULTIVO DE OUTROS CEREAIS NÃO ESPECIFICADOS AN...,0111-3/99
4,112101,CULTIVO DE ALGODÃO HERBÁCEO,0112-1/01
...,...,...,...
495,2920402,FABRICAÇÃO DE MOTORES PARA CAMINHÕES E ÔNIBUS,2920-4/02
496,2930101,"FABRICAÇÃO DE CABINES, CARROCERIAS E REBOQUES ...",2930-1/01
497,2930102,FABRICAÇÃO DE CARROCERIAS PARA ÔNIBUS,2930-1/02
498,2930103,"FABRICAÇÃO DE CABINES, CARROCERIAS E REBOQUES ...",2930-1/03


In [7]:
df.to_excel('TABELA_DESCRICAO_CNAE.xlsx',engine='openpyxl', sheet_name='Lista Cnaes', index=False, author='Gustavo Henrique Lopes Martins')

TypeError: NDFrame.to_excel() got an unexpected keyword argument 'author'